In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
import zipfile
from scipy.stats import norm
from datetime import datetime
import os

In [29]:
def calculate_volatility(ticker_symbols, start_date, end_date):
    data_dict = {}
    for ticker_symbol in ticker_symbols:
        # Fetch historical data
        data = yf.download(ticker_symbol, start=start_date, end=end_date)

        # Calculate daily returns
        data['Returns'] = data['Adj Close'].pct_change()

        # Calculate volatility
        volatility = data['Returns'].std() * (252 ** 0.5)  # 252 trading days in a year
        closing_price = data['Adj Close'].iloc[-1]  # Latest closing price

        data_dict[ticker_symbol] = {'Volatility': volatility, 'Closing Price': closing_price}

    return data_dict

In [24]:
# Black-Scholes option pricing formula for European options
def black_scholes(option_type, S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'C':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'P':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return option_price

In [25]:
def fetch_options_data_call(ticker_symbol, expiration_date, filepath):
    try:
        ticker = yf.Ticker(ticker_symbol) #yahoo ticker in
        options_data = ticker.option_chain(expiration_date) #option chain in
        
        options_df = options_data.calls
        
        # Calculate option price, bid difference, and ask difference
        #gets volatility of all tickers
        data_dict = calculate_volatility([ticker_symbol], pd.Timestamp.now() - pd.DateOffset(years=1), pd.Timestamp.now()) 
        volatility = data_dict[ticker_symbol]['Volatility'] #gets vol of just this ticker
        closing_price = data_dict[ticker_symbol]['Closing Price'] #gets price of the ticker
        
        for index, row in options_df.iterrows():
            option_type = row['contractSymbol'][len(ticker_symbol)+6] #gets the option type from the row
            strike_price = row['strike'] #gets the strike price from the row
            #gets the maturity from the row
            maturity_date = datetime.strptime(row['contractSymbol'][len(ticker_symbol):len(ticker_symbol)+6], '%y%m%d') 
            maturity = (maturity_date - pd.Timestamp.now()).days / 365
            
            #prices and gets difference between option prices
            risk_free_rate = 0.0525
            option_price = black_scholes(option_type, closing_price, strike_price, maturity, risk_free_rate, volatility)
            bid_diff = row['bid'] - option_price
            ask_diff = row['ask'] - option_price
            
           
            options_df.at[index, 'Option Type'] = option_type 
            options_df.at[index, 'Strike Price'] = strike_price 
            options_df.at[index, 'Bid Difference'] = bid_diff
            options_df.at[index, 'Ask Difference'] = ask_diff

        
        # Add the printed information to the DataFrame
        options_df['Ticker'] = ticker_symbol
        options_df['Expiration Date'] = expiration_date
        
        # Export options data to CSV
        options_df.to_csv(filepath, index=False)
        print(f'Successfully fetched options data for {ticker_symbol}')
    except ValueError as e:
        print(f'Error fetching options data for {ticker_symbol}: {e}')

In [26]:
def fetch_options_data_put(ticker_symbol, expiration_date, filepath):
    try:
        ticker = yf.Ticker(ticker_symbol) #yahoo ticker in
        options_data = ticker.option_chain(expiration_date) #option chain in

        options_df = options_data.puts
        
        # Combine calls and puts dataframes
        #options_df = pd.concat([calls_df, puts_df], axis=0) 
   
        # Calculate option price, bid difference, and ask difference
        data_dict = calculate_volatility([ticker_symbol], pd.Timestamp.now() - pd.DateOffset(years=1), pd.Timestamp.now()) #gets volatility of all tickers
        volatility = data_dict[ticker_symbol]['Volatility'] #gets vol of just this ticker
        closing_price = data_dict[ticker_symbol]['Closing Price'] #gets price of the ticker
        
        for index, row in options_df.iterrows():
            option_type = row['contractSymbol'][len(ticker_symbol)+6] #gets the option type from the row
            strike_price = row['strike'] #gets the strike price from the row
            maturity_date = datetime.strptime(row['contractSymbol'][len(ticker_symbol):len(ticker_symbol)+6], '%y%m%d') #gets the maturity from the row
            maturity = (maturity_date - pd.Timestamp.now()).days / 365
            
            #prices and gets difference between option prices
            risk_free_rate = 0.0525
            option_price = black_scholes(option_type, closing_price, strike_price, maturity, risk_free_rate, volatility)
            bid_diff = row['bid'] - option_price
            ask_diff = row['ask'] - option_price
            
            options_df.at[index, 'Option Type'] = option_type 
            options_df.at[index, 'Strike Price'] = strike_price 
            options_df.at[index, 'Bid Difference'] = bid_diff
            options_df.at[index, 'Ask Difference'] = ask_diff
 
        # Add the printed information to the DataFrame
        options_df['Ticker'] = ticker_symbol
        options_df['Expiration Date'] = expiration_date
        
        # Export options data to CSV
        options_df.to_csv(filepath, index=False)
        print(f'Successfully fetched options data for {ticker_symbol}')
    except ValueError as e:
        print(f'Error fetching options data for {ticker_symbol}: {e}')

In [27]:
# Load the ticker symbols from the Excel file
ticker_symbols_df = pd.read_excel("tickers.xlsx")

In [28]:
# Specify the expiration date of the options contracts
expiration_date = '2024-05-17'

In [19]:
# Loop through each ticker symbol and fetch its options data
for ticker_symbol in ticker_symbols_df['Symbol']:
    filepath_call = f'options_data_call_{ticker_symbol}.csv'
     filepath_put = f'options_data_put_{ticker_symbol}.csv'

    fetch_options_data_call(ticker_symbol, expiration_date, filepath_call)
    fetch_options_data_put(ticker_symbol, expiration_date, filepath_put)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Successfully fetched options data for MMM
Successfully fetched options data for MMM


[*********************100%%**********************]  1 of 1 completed


Successfully fetched options data for AOS


[*********************100%%**********************]  1 of 1 completed


...

Successfully fetched options data for ZBRA
Successfully fetched options data for ZBRA


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Successfully fetched options data for ZBH
Successfully fetched options data for ZBH


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Successfully fetched options data for ZTS
Successfully fetched options data for ZTS


In [35]:

def combine_csv_files(file1, file2, output_file):
    filepath_call =  f'options_data_call_{ticker_symbol}.csv' #  file path template
    filepath_put =  f'options_data_put_{ticker_symbol}.csv'
   
    #Combine two CSV files into one and save the combined data into a new CSV file.
    
    # Read data from both CSV files
    df1 = pd.read_csv(f'{filepath_call}')
    df2 = pd.read_csv(f'{filepath_put}')
    
    # Concatenate the data
    combined_df = pd.concat([df1, df2], ignore_index=True)
    
    # Save the combined data to a new CSV file
    combined_df.to_csv(output_file, index=False)

# combine_csv_files(file1, file2, output_file)
for ticker_symbol in ticker_symbols_df['Symbol']:
    filepath_call = f'options_data_call_{ticker_symbol}.csv'
    filepath_put = f'options_data_put_{ticker_symbol}.csv'
    output_file = f'options_data_combined_{ticker_symbol}.csv'

    # Fetch call and put options data
    fetch_options_data_call(ticker_symbol, expiration_date, filepath_call)
    fetch_options_data_put(ticker_symbol, expiration_date, filepath_put)

    # Combine call and put option CSV files
    combine_csv_files(filepath_call, filepath_put, output_file)
    
# remove individual call and put option files after combining
    os.remove(filepath_call)
    os.remove(filepath_put)
    

'\ndef combine_csv_files(file1, file2, output_file):\n    filepath_call =  f\'options_data_call_{ticker_symbol}.csv\' #  file path template\n    filepath_put =  f\'options_data_put_{ticker_symbol}.csv\'\n    """\n    Combine two CSV files into one and save the combined data into a new CSV file.\n    """\n    # Read data from both CSV files\n    df1 = pd.read_csv(f\'{filepath_call}\')\n    df2 = pd.read_csv(f\'{filepath_put}\')\n    \n    # Concatenate the data\n    combined_df = pd.concat([df1, df2], ignore_index=True)\n    \n    # Save the combined data to a new CSV file\n    combined_df.to_csv(output_file, index=False)\n\n# combine_csv_files(file1, file2, output_file)\nfor ticker_symbol in ticker_symbols_df[\'Symbol\']:\n    filepath_call = f\'options_data_call_{ticker_symbol}.csv\'\n    filepath_put = f\'options_data_put_{ticker_symbol}.csv\'\n    output_file = f\'options_data_combined_{ticker_symbol}.csv\'\n\n    # Fetch call and put options data\n    fetch_options_data_call(ticker